In [1]:
import numpy as np
import pandas as pd
pd.set_option("display.max_columns", None)
pd.set_option("display.max_rows", None)
pd.set_option("display.max_colwidth", None)
pd.set_option("display.width", None)
df_sic_peers = pd.read_csv("df_sic_peers.csv")
df_sic_peers[["ticker", "datekey"]][df_sic_peers["ticker"] == "ACI"]
# Per manual inspection, ACI's price series starts around the end of
# 6/2020, but there are financial statement obs going back to 2018.
# I think this is because of the data filed with the S1 statement
# for the IPO.

len(df_sic_peers["ticker"].unique())
df_sic_peers["ticker"].unique()
df_prices = pd.read_csv("prices.csv")
len(df_prices["ticker"].unique())
# There are 19 unique tickers in the financial statement data,
# but 20 in the price data.

df_prices["ticker"].unique()
# Below identifies the ticker in the price data that is not in the
# financial statement data.

for i in df_prices["ticker"].unique():
    if i not in df_sic_peers["ticker"].unique():
        print(i)
# Per manual inspection of the SF1 table, CBD only has ART, ARY, MRT and MRY
# for dimension so they weren't included in the original data pull.
# The original data query had dimension="ARQ"

df_prices[df_prices["ticker"] == "ACI"]
df_prices[
    (df_prices["ticker"] == "ACI")
    & (df_prices["date"] >= "2018-05-07")
    & (df_prices["date"] <= "2020-06-24")
]
# Above shows no price data before the end of June, 2020.

df_sic_peers.head()
# Below identifies SEC filing dates without a stock price date. At first,
# I thought these would all be cases of companies filing on holidays or
# weekends, but after manual inspection it appears that most of these
# are related to IPO's. See notes and further inspection below the loop.
# First, itererate through the dataframe based on the unique tickers.

for i in df_sic_peers["ticker"].unique():
    df_temp_sic = df_sic_peers[df_sic_peers["ticker"] == i]
    df_temp_prices = df_prices[df_prices["ticker"] == i]
    # Create two lists, one that contains the dates from the SEC fillings
    # and the other that contains dates from the stock prices series.
    lister = []
    lister_2 = []
    for j in df_temp_sic["datekey"]:
        lister.append(j)
    for q in df_temp_prices["date"]:
        lister_2.append(q)
    # Identify financial statement dates without stock price data. Also,
    # see if shifting the SEC reporting dates forward by a day or two and
    # backward by a day or two result in mergeable dates. Note that this
    # only worked for GO and FWMHQ around their IPOs. This is further
    # examined below the loop. GO's first trading day was 6/20/19
    # and FWMHQ's first trading day was 4/15/13. The loops further
    # condition on pd.dattime().weekday() values which are 0 for Monday
    # and 6 for Sunday. .weekday() values in the data are [0-4].
    for z in lister:
        if z not in lister_2:
            print("Non-trading date fillings:", i, z)
            print("Day of the week is:", i, z, pd.to_datetime(z).weekday())
            print(
                str(pd.to_datetime(z).date()),
                str(pd.to_datetime(z).date() + pd.Timedelta("1 day")),
            )
            if pd.to_datetime(z).weekday() <= 3:
                print(
                    "Plus one day:",
                    i,
                    z,
                    str(pd.to_datetime(z).date() + pd.Timedelta("1 day")),
                    str(pd.to_datetime(z).date() + pd.Timedelta("1 day"))
                    in lister_2,
                )
            if pd.to_datetime(z).weekday() <= 3:
                print(
                    "Minus one day:",
                    i,
                    z,
                    str(pd.to_datetime(z).date() - pd.Timedelta("1 day")),
                    str(pd.to_datetime(z).date() - pd.Timedelta("1 day"))
                    in lister_2,
                )
            if pd.to_datetime(z).weekday() <= 3:
                print(
                    "Plus two days:",
                    i,
                    z,
                    str(pd.to_datetime(z).date() + pd.Timedelta("2 days")),
                    str(pd.to_datetime(z).date() + pd.Timedelta("2 days"))
                    in lister_2,
                )
            if pd.to_datetime(z).weekday() <= 3:
                print(
                    "Minus two days:",
                    i,
                    z,
                    str(pd.to_datetime(z).date() - pd.Timedelta("2 days")),
                    str(pd.to_datetime(z).date() - pd.Timedelta("2 days"))
                    in lister_2,
                )
            if (pd.to_datetime(z).weekday() > 3) & (
                pd.to_datetime(z).weekday() <= 5
            ):
                print(
                    "Plus one day:",
                    i,
                    z,
                    str(pd.to_datetime(z).date() + pd.Timedelta("1 day")),
                    str(pd.to_datetime(z).date() + pd.Timedelta("1 day"))
                    in lister_2,
                )
                print(
                    "Minus one day:",
                    i,
                    z,
                    str(pd.to_datetime(z).date() - pd.Timedelta("1 day")),
                    str(pd.to_datetime(z).date() - pd.Timedelta("1 day"))
                    in lister_2,
                )
                print(
                    "Plus two days:",
                    i,
                    z,
                    str(pd.to_datetime(z).date() + pd.Timedelta("2 days")),
                    str(pd.to_datetime(z).date() + pd.Timedelta("2 days"))
                    in lister_2,
                )
                print(
                    "Minus two days:",
                    i,
                    z,
                    str(pd.to_datetime(z).date() - pd.Timedelta("2 days")),
                    str(pd.to_datetime(z).date() - pd.Timedelta("2 days"))
                    in lister_2,
                )



CBD
Non-trading date fillings: SWY 2015-03-04
Day of the week is: SWY 2015-03-04 2
2015-03-04 2015-03-05
Plus one day: SWY 2015-03-04 2015-03-05 False
Minus one day: SWY 2015-03-04 2015-03-03 False
Plus two days: SWY 2015-03-04 2015-03-06 False
Minus two days: SWY 2015-03-04 2015-03-02 False
Non-trading date fillings: SFM 2013-07-29
Day of the week is: SFM 2013-07-29 0
2013-07-29 2013-07-30
Plus one day: SFM 2013-07-29 2013-07-30 False
Minus one day: SFM 2013-07-29 2013-07-28 False
Plus two days: SFM 2013-07-29 2013-07-31 False
Minus two days: SFM 2013-07-29 2013-07-27 False
Non-trading date fillings: SFM 2013-07-22
Day of the week is: SFM 2013-07-22 0
2013-07-22 2013-07-23
Plus one day: SFM 2013-07-22 2013-07-23 False
Minus one day: SFM 2013-07-22 2013-07-21 False
Plus two days: SFM 2013-07-22 2013-07-24 False
Minus two days: SFM 2013-07-22 2013-07-20 False
Non-trading date fillings: NGVC 2012-07-20
Day of the week is: NGVC 2012-07-20 4
2012-07-20 2012-07-21
Plus one day: NGVC 2012-07

In [2]:
df_prices[
    (df_prices["ticker"] == "SWY")
    & (df_prices["date"] >= "2015-03-01")
    & (df_prices["date"] <= "2015-03-10")
]

df_prices[df_prices["ticker"] == "SWY"]
df_prices[
    (df_prices["ticker"] == "GO")
    & (df_prices["date"] >= "2019-06-01")
    & (df_prices["date"] <= "2019-06-30")
]

df_prices[
    (df_prices["ticker"] == "FWMHQ")
    & (df_prices["date"] >= "2013-04-01")
    & (df_prices["date"] <= "2013-04-30")
]

df_sic_peers.head()


,ticker,dimension,calendardate,datekey,reportperiod,lastupdated,accoci,assets,assetsavg,assetsc,assetsnc,assetturnover,bvps,capex,cashneq,cashnequsd,cor,consolinc,currentratio,de,debt,debtc,debtnc,debtusd,deferredrev,depamor,deposits,divyield,dps,ebit,ebitda,ebitdamargin,ebitdausd,ebitusd,ebt,eps,epsdil,epsusd,equity,equityavg,equityusd,ev,evebit,evebitda,fcf,fcfps,fxusd,gp,grossmargin,intangibles,intexp,invcap,invcapavg,inventory,investments,investmentsc,investmentsnc,liabilities,liabilitiesc,liabilitiesnc,marketcap,ncf,ncfbus,ncfcommon,ncfdebt,ncfdiv,ncff,ncfi,ncfinv,ncfo,ncfx,netinc,netinccmn,netinccmnusd,netincdis,netincnci,netmargin,opex,opinc,payables,payoutratio,pb,pe,pe1,ppnenet,prefdivis,price,ps,ps1,receivables,retearn,revenue,revenueusd,rnd,roa,roe,roic,ros,sbcomp,sgna,sharefactor,sharesbas,shareswa,shareswadil,sps,tangibles,taxassets,taxexp,taxliabilities,tbvps,workingcapital
0,WMK,ARQ,2021-03-31,2021-05-06,2021-03-27,2021-05-06,2512000.0,1.784923e+09,NaN,586996000.0,1.197927e+09,NaN,43.172,-37014000.0,101217000.0,101217000.0,738950000.0,24255000.0,1.970,0.537,203899000.0,39352000.0,164547000.0,203899000.0,7947000.0,25359000.0,0.0,0.023,0.31,32670000.0,58029000.0,0.058,58029000.0,32670000.0,32670000.0,0.90,0.90,0.90,1.161257e+09,NaN,1.161257e+09,1.562729e+09,10.0,5.985,-22077000.0,-0.821,1.0,267390000.0,0.266,69738000.0,0.0,1.519828e+09,NaN,276733000.0,114727000.0,114727000.0,0.0,623666000.0,298039000.0,325627000.0,1.460047e+09,-35395000.0,0.0,0.0,0.0,-8339000.0,-8339000.0,-41993000.0,-4962000.0,14937000.0,0.0,24255000.0,24255000.0,24255000.0,0.0,0.0,0.024,235556000.0,31834000.0,184536000.0,0.344,1.257,12.534,12.565,1.128189e+09,0.0,54.28,0.353,0.353,44598000.0,1.299653e+09,1.006340e+09,1.006340e+09,0.0,NaN,NaN,NaN,NaN,0.0,235556000.0,1.0,26898443.0,26898443.0,26898443.0,37.413,1.715185e+09,0.0,8415000.0,115435000.0,63.765,288957000.0
1,WMK,ARQ,2020-12-31,2021-03-11,2020-12-26,2021-05-06,3286000.0,1.820421e+09,NaN,626202000.0,1.194219e+09,NaN,42.609,-47255000.0,136612000.0,136612000.0,756623000.0,19420000.0,1.808,0.588,208229000.0,39165000.0,169064000.0,208229000.0,10583000.0,25367000.0,0.0,0.021,0.31,27666000.0,53033000.0,0.052,53033000.0,27666000.0,27666000.0,0.72,0.72,0.72,1.146115e+09,NaN,1.146115e+09,1.658087e+09,10.0,6.303,36554000.0,1.359,1.0,269068000.0,0.262,69889000.0,0.0,1.475880e+09,NaN,269024000.0,111855000.0,111855000.0,0.0,674306000.0,346269000.0,328037000.0,1.586470e+09,19204000.0,0.0,0.0,0.0,-8338000.0,-8338000.0,-56267000.0,-9012000.0,83809000.0,0.0,19420000.0,19420000.0,19420000.0,0.0,0.0,0.019,242126000.0,26942000.0,221566000.0,0.431,1.384,13.341,13.344,1.124330e+09,0.0,58.98,0.386,0.386,56760000.0,1.283737e+09,1.025691e+09,1.025691e+09,0.0,NaN,NaN,NaN,NaN,0.0,242126000.0,1.0,26898443.0,26898443.0,26898443.0,38.132,1.750532e+09,0.0,8246000.0,108462000.0,65.079,279933000.0
2,WMK,ARQ,2020-09-30,2020-11-05,2020-09-26,2021-05-06,2959000.0,1.756733e+09,NaN,586745000.0,1.169988e+09,NaN,42.185,-33812000.0,117408000.0,117408000.0,728054000.0,31336000.0,1.952,0.548,210422000.0,39297000.0,171125000.0,210422000.0,6539000.0,26209000.0,0.0,0.027,0.00,43478000.0,69687000.0,0.070,69687000.0,43478000.0,43478000.0,1.16,1.16,1.16,1.134706e+09,NaN,1.134706e+09,1.349171e+09,8.0,5.235,21294000.0,0.792,1.0,274333000.0,0.274,70162000.0,0.0,1.478923e+09,NaN,265990000.0,102449000.0,102449000.0,0.0,622027000.0,300662000.0,321365000.0,1.256157e+09,8748000.0,0.0,0.0,0.0,-8339000.0,-8339000.0,-38019000.0,-4039000.0,55106000.0,0.0,31336000.0,31336000.0,31336000.0,0.0,0.0,0.031,231527000.0,42806000.0,188714000.0,0.000,1.107,10.611,10.638,1.099826e+09,0.0,46.70,0.315,0.315,50726000.0,1.272655e+09,1.002387e+09,1.002387e+09,0.0,NaN,NaN,NaN,NaN,0.0,231527000.0,1.0,26898443.0,26898443.0,26898443.0,37.266,1.686571e+09,0.0,12142000.0,109646000.0,62.701,286083000.0
3,WMK,ARQ,2020-06-30,2020-08-05,2020-06-27,2021-05-06,2598000.0,1.739130e+09,NaN,578379000.0,1.160751e+09,NaN,41.316,-24767000.0,108660000.0,108660000.0,805816000.0,414720

In [3]:
df_sic_peers.loc[
    (df_sic_peers["datekey"] == "2019-06-18")
    & (df_sic_peers["ticker"] == "GO")
]

df_sic_peers.loc[
    (df_sic_peers["datekey"] == "2019-06-18")
    & (df_sic_peers["ticker"] == "GO"),
    "datekey",
] = "2019-06-20"

df_sic_peers.loc[
    (df_sic_peers["datekey"] == "2019-06-20")
    & (df_sic_peers["ticker"] == "GO")
]

df_sic_peers.loc[
    (df_sic_peers["datekey"] == "2013-04-16")
    & (df_sic_peers["ticker"] == "FWMHQ")
]

df_sic_peers.loc[
    (df_sic_peers["datekey"] == "2013-04-16")
    & (df_sic_peers["ticker"] == "FWMHQ"),
    "datekey",
] = "2013-04-17"

df_sic_peers.loc[
    (df_sic_peers["datekey"] == "2013-04-17")
    & (df_sic_peers["ticker"] == "FWMHQ")
]



,ticker,dimension,calendardate,datekey,reportperiod,lastupdated,accoci,assets,assetsavg,assetsc,assetsnc,assetturnover,bvps,capex,cashneq,cashnequsd,cor,consolinc,currentratio,de,debt,debtc,debtnc,debtusd,deferredrev,depamor,deposits,divyield,dps,ebit,ebitda,ebitdamargin,ebitdausd,ebitusd,ebt,eps,epsdil,epsusd,equity,equityavg,equityusd,ev,evebit,evebitda,fcf,fcfps,fxusd,gp,grossmargin,intangibles,intexp,invcap,invcapavg,inventory,investments,investmentsc,investmentsnc,liabilities,liabilitiesc,liabilitiesnc,marketcap,ncf,ncfbus,ncfcommon,ncfdebt,ncfdiv,ncff,ncfi,ncfinv,ncfo,ncfx,netinc,netinccmn,netinccmnusd,netincdis,netincnci,netmargin,opex,opinc,payables,payoutratio,pb,pe,pe1,ppnenet,prefdivis,price,ps,ps1,receivables,retearn,revenue,revenueusd,rnd,roa,roe,roic,ros,sbcomp,sgna,sharefactor,sharesbas,shareswa,shareswadil,sps,tangibles,taxassets,taxexp,taxliabilities,tbvps,workingcapital
361,FWMHQ,ARQ,2012-12-31,2013-04-17,2012-12-30,2019-06-07,0.0,339178000.0,NaN,75015000.0,264163000.0,NaN,-5.324,NaN,29172000.0,29172000.0,NaN,NaN,1.273,-1.513,254627000.0,2600000.0,252027000.0,254627000.0,0.0,NaN,0.0,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-219570000.0,NaN,-219570000.0,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,121146000.0,NaN,384581000.0,NaN,24062000.0,0.0,0.0,0.0,332215000.0,58906000.0,273309000.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,38174000.0,NaN,NaN,NaN,NaN,120352000.0,NaN,NaN,NaN,NaN,8928000.0,-219570000.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,41238260.0,41238260.0,NaN,NaN,218032000.0,4856000.0,NaN,0.0,5.287,16109000.0


In [4]:
df_sic_peers.info(verbose=True)
df_prices.info()
# Convert date variables to datetime.

df_sic_peers["datekey"] = pd.to_datetime(df_sic_peers["datekey"])
df_prices["date"] = pd.to_datetime(df_prices["date"])
df_sic_peers.rename(columns={"datekey": "date"}, inplace=True)
# Merge

df = pd.merge(df_prices, df_sic_peers, on=["ticker", "date"], how="left")
df.head()
len(df)
len(df_prices)
len(df_sic_peers)
# Sort by ticker and datekey

df.sort_values(by=["ticker", "date"], ascending=[True, True], inplace=True)
df.head()
df.reset_index(drop=True, inplace=True)
df.head()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 384 entries, 0 to 383
Data columns (total 111 columns):
 #   Column          Dtype  
---  ------          -----  
 0   ticker          object 
 1   dimension       object 
 2   calendardate    object 
 3   datekey         object 
 4   reportperiod    object 
 5   lastupdated     object 
 6   accoci          float64
 7   assets          float64
 8   assetsavg       float64
 9   assetsc         float64
 10  assetsnc        float64
 11  assetturnover   float64
 12  bvps            float64
 13  capex           float64
 14  cashneq         float64
 15  cashnequsd      float64
 16  cor             float64
 17  consolinc       float64
 18  currentratio    float64
 19  de              float64
 20  debt            float64
 21  debtc           float64
 22  debtnc          float64
 23  debtusd         float64
 24  deferredrev     float64
 25  depamor         float64
 26  deposits        float64
 27  divyield        float64
 28  dps             flo

,ticker,date,open,high,low,close,volume,closeadj,closeunadj,lastupdated_x,dimension,calendardate,reportperiod,lastupdated_y,accoci,assets,assetsavg,assetsc,assetsnc,assetturnover,bvps,capex,cashneq,cashnequsd,cor,consolinc,currentratio,de,debt,debtc,debtnc,debtusd,deferredrev,depamor,deposits,divyield,dps,ebit,ebitda,ebitdamargin,ebitdausd,ebitusd,ebt,eps,epsdil,epsusd,equity,equityavg,equityusd,ev,evebit,evebitda,fcf,fcfps,fxusd,gp,grossmargin,intangibles,intexp,invcap,invcapavg,inventory,investments,investmentsc,investmentsnc,liabilities,liabilitiesc,liabilitiesnc,marketcap,ncf,ncfbus,ncfcommon,ncfdebt,ncfdiv,ncff,ncfi,ncfinv,ncfo,ncfx,netinc,netinccmn,netinccmnusd,netincdis,netincnci,netmargin,opex,opinc,payables,payoutratio,pb,pe,pe1,ppnenet,prefdivis,price,ps,ps1,receivables,retearn,revenue,revenueusd,rnd,roa,roe,roic,ros,sbcomp,sgna,sharefactor,sharesbas,shareswa,shareswadil,sps,tangibles,taxassets,taxexp,taxliabilities,tbvps,workingcapital
0,ACI,2020-06-26,15.50,16.50,15.31,15.45,37216510.0,15.185,15.45,2021-04-26,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,ACI,2020-06-29,15.89,16.01,15.50,15.57,7349059.0,15.303,15.57,2021-04-26,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,ACI,2020-06-30,15.57,15.90,15.45,15.77,5030640.0,15.500,15.77,2021-04-26,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,ACI,2020-07-01,15.79,15.89,15.55,15.76,3400498.0,15.490,15.76,2021-04-26,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,ACI,2020-07-02,15.82,15.89,15.60,15.81,7190929.0,15.539,15.81,2021-04-26,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [5]:
lister = []
def cumRet(df, window):
    """ This function calculates the return for the period of t-1 from the
    EDGAR file date through t + window. New dataframes for each ticker
    are then stored into a list, lister. These will be merged in later
    with the dataframe, df, that has both price and financial
    statement data.  """
    for i, t in enumerate(df["ticker"].unique()):
        df_temp = df[df["ticker"] == t].copy()
        # df_temp['cum_ret']= np.nan
        df_temp.loc[:, "cum_ret"] = np.nan
        cut_offs = []
        for j in df_temp["calendardate"][
            df_temp["calendardate"].notnull()
        ].index:
            cut_offs.append(j)
        for x in cut_offs:
            # start the window from the day before earnings are announced.
            # this is to try and deal with companies that may release earnings
            # either after markets close on a given day, before
            # markets open on a given day, or during a trading day.
            # Below modifies starting index for GO and FWMHQ who
            # have no price observations before the IPO filings.
            if x - 1 not in df_temp.index:
                start_index = df_temp.index.min()
            else:
                start_index = x - 1
            # if there are not t-1 plus 16 trading days after a
            # given announcement, use the last trading day for the
            # company in the dataframe.
            if x + window not in df_temp.index:
                end_index = df_temp.index.max()
            else:
                end_index = start_index + window
            print(
                "ticker is",
                t,
                "start_index",
                start_index,
                "end_index",
                end_index,
            )
            print(
                "start_price",
                df_temp["close"].loc[start_index],
                "end_price",
                df_temp["close"].loc[end_index],
            )
            # modify the start_index by adding one to line up with
            # the actual earnings report release date.
            df_temp["cum_ret"].loc[start_index + 1] = (
                df_temp["close"].loc[end_index]
                - df_temp["close"].loc[start_index]
            ) / df_temp["close"].loc[start_index]
        print("appending", df_temp["ticker"].iloc[0])
        lister.append(df_temp[["ticker", "date", "cum_ret"]])



In [6]:
cumRet(df, 16)


ticker is ACI start_index 25 end_index 41
start_price 14.83 end_price 14.59
ticker is ACI start_index 80 end_index 96
start_price 15.22 end_price 14.98
ticker is ACI start_index 137 end_index 153
start_price 17.1 end_price 16.71
ticker is ACI start_index 209 end_index 217
start_price 18.11 end_price 19.18
appending ACI
ticker is ARDNA start_index 240 end_index 256
start_price 82.6 end_price 89.0
ticker is ARDNA start_index 304 end_index 320
start_price 95.88 end_price 111.99
ticker is ARDNA start_index 388 end_index 404
start_price 98.31 end_price 100.15
ticker is ARDNA start_index 429 end_index 445
start_price 98.65 end_price 107.54
ticker is ARDNA start_index 493 end_index 509
start_price 132.78 end_price 134.6
ticker is ARDNA start_index 557 end_index 573
start_price 129.85 end_price 129.99
appending ARDNA
ticker is ASAI start_index 664 end_index 671
start_price 14.81 end_price 16.86
appending ASAI
appending CBD
ticker is FWMHQ start_index 2899 end_index 2915
start_price 17.35 end_p

In [7]:
len(lister)


20

In [8]:
combined_df = pd.concat([i for i in lister])
len(combined_df)
combined_df.head()


,ticker,date,cum_ret
0,ACI,2020-06-26,NaN
1,ACI,2020-06-29,NaN
2,ACI,2020-06-30,NaN
3,ACI,2020-07-01,NaN
4,ACI,2020-07-02,NaN


In [9]:
df_fin = pd.merge(df_sic_peers, combined_df, on=["ticker", "date"], how="left")


In [10]:
df_fin.shape
df_fin.head()


,ticker,dimension,calendardate,date,reportperiod,lastupdated,accoci,assets,assetsavg,assetsc,assetsnc,assetturnover,bvps,capex,cashneq,cashnequsd,cor,consolinc,currentratio,de,debt,debtc,debtnc,debtusd,deferredrev,depamor,deposits,divyield,dps,ebit,ebitda,ebitdamargin,ebitdausd,ebitusd,ebt,eps,epsdil,epsusd,equity,equityavg,equityusd,ev,evebit,evebitda,fcf,fcfps,fxusd,gp,grossmargin,intangibles,intexp,invcap,invcapavg,inventory,investments,investmentsc,investmentsnc,liabilities,liabilitiesc,liabilitiesnc,marketcap,ncf,ncfbus,ncfcommon,ncfdebt,ncfdiv,ncff,ncfi,ncfinv,ncfo,ncfx,netinc,netinccmn,netinccmnusd,netincdis,netincnci,netmargin,opex,opinc,payables,payoutratio,pb,pe,pe1,ppnenet,prefdivis,price,ps,ps1,receivables,retearn,revenue,revenueusd,rnd,roa,roe,roic,ros,sbcomp,sgna,sharefactor,sharesbas,shareswa,shareswadil,sps,tangibles,taxassets,taxexp,taxliabilities,tbvps,workingcapital,cum_ret
0,WMK,ARQ,2021-03-31,2021-05-06,2021-03-27,2021-05-06,2512000.0,1.784923e+09,NaN,586996000.0,1.197927e+09,NaN,43.172,-37014000.0,101217000.0,101217000.0,738950000.0,24255000.0,1.970,0.537,203899000.0,39352000.0,164547000.0,203899000.0,7947000.0,25359000.0,0.0,0.023,0.31,32670000.0,58029000.0,0.058,58029000.0,32670000.0,32670000.0,0.90,0.90,0.90,1.161257e+09,NaN,1.161257e+09,1.562729e+09,10.0,5.985,-22077000.0,-0.821,1.0,267390000.0,0.266,69738000.0,0.0,1.519828e+09,NaN,276733000.0,114727000.0,114727000.0,0.0,623666000.0,298039000.0,325627000.0,1.460047e+09,-35395000.0,0.0,0.0,0.0,-8339000.0,-8339000.0,-41993000.0,-4962000.0,14937000.0,0.0,24255000.0,24255000.0,24255000.0,0.0,0.0,0.024,235556000.0,31834000.0,184536000.0,0.344,1.257,12.534,12.565,1.128189e+09,0.0,54.28,0.353,0.353,44598000.0,1.299653e+09,1.006340e+09,1.006340e+09,0.0,NaN,NaN,NaN,NaN,0.0,235556000.0,1.0,26898443.0,26898443.0,26898443.0,37.413,1.715185e+09,0.0,8415000.0,115435000.0,63.765,288957000.0,0.019757
1,WMK,ARQ,2020-12-31,2021-03-11,2020-12-26,2021-05-06,3286000.0,1.820421e+09,NaN,626202000.0,1.194219e+09,NaN,42.609,-47255000.0,136612000.0,136612000.0,756623000.0,19420000.0,1.808,0.588,208229000.0,39165000.0,169064000.0,208229000.0,10583000.0,25367000.0,0.0,0.021,0.31,27666000.0,53033000.0,0.052,53033000.0,27666000.0,27666000.0,0.72,0.72,0.72,1.146115e+09,NaN,1.146115e+09,1.658087e+09,10.0,6.303,36554000.0,1.359,1.0,269068000.0,0.262,69889000.0,0.0,1.475880e+09,NaN,269024000.0,111855000.0,111855000.0,0.0,674306000.0,346269000.0,328037000.0,1.586470e+09,19204000.0,0.0,0.0,0.0,-8338000.0,-8338000.0,-56267000.0,-9012000.0,83809000.0,0.0,19420000.0,19420000.0,19420000.0,0.0,0.0,0.019,242126000.0,26942000.0,221566000.0,0.431,1.384,13.341,13.344,1.124330e+09,0.0,58.98,0.386,0.386,56760000.0,1.283737e+09,1.025691e+09,1.025691e+09,0.0,NaN,NaN,NaN,NaN,0.0,242126000.0,1.0,26898443.0,26898443.0,26898443.0,38.132,1.750532e+09,0.0,8246000.0,108462000.0,65.079,279933000.0,-0.033475
2,WMK,ARQ,2020-09-30,2020-11-05,2020-09-26,2021-05-06,2959000.0,1.756733e+09,NaN,586745000.0,1.169988e+09,NaN,42.185,-33812000.0,117408000.0,117408000.0,728054000.0,31336000.0,1.952,0.548,210422000.0,39297000.0,171125000.0,210422000.0,6539000.0,26209000.0,0.0,0.027,0.00,43478000.0,69687000.0,0.070,69687000.0,43478000.0,43478000.0,1.16,1.16,1.16,1.134706e+09,NaN,1.134706e+09,1.349171e+09,8.0,5.235,21294000.0,0.792,1.0,274333000.0,0.274,70162000.0,0.0,1.478923e+09,NaN,265990000.0,102449000.0,102449000.0,0.0,622027000.0,300662000.0,321365000.0,1.256157e+09,8748000.0,0.0,0.0,0.0,-8339000.0,-8339000.0,-38019000.0,-4039000.0,55106000.0,0.0,31336000.0,31336000.0,31336000.0,0.0,0.0,0.031,231527000.0,42806000.0,188714000.0,0.000,1.107,10.611,10.638,1.099826e+09,0.0,46.70,0.315,0.315,50726000.0,1.272655e+09,1.002387e+09,1.002387e+09,0.0,NaN,NaN,NaN,NaN,0.0,231527000.0,1.0,26898443.0,26898443.0,26898443.0,37.266,1.686571e+09,0.0,12142000.0,109646000.0,62.701,286083000.0,0.032794
3,WMK,ARQ,2020-06-30,2020-08-05,2020-06-27,2021-05-06,2598000.0,1.739130e+09,NaN,578379000.0,1.160751e+09,NaN,41.316,-24767000.0,108660000

In [11]:
df_fin.sort_values(by=["ticker", "date"], ascending=[True, True], inplace=True)


In [12]:
df_fin[["ticker", "date", "cum_ret"]]


,ticker,date,cum_ret
383,ACI,2018-05-11,NaN
382,ACI,2018-07-25,NaN
381,ACI,2018-10-18,NaN
380,ACI,2019-01-15,NaN
379,ACI,2019-04-24,NaN
378,ACI,2019-07-24,NaN
377,ACI,2019-10-16,NaN
376,ACI,2020-01-08,NaN
375,ACI,2020-05-13,NaN
374,ACI,2020-06-18,NaN


In [13]:
len(df_fin["ticker"].unique())


19

In [14]:
df_fin["ticker"].unique()


array(['ACI', 'ARDNA', 'ASAI', 'FWMHQ', 'GO', 'HTSI', 'IFMK', 'IMKTA',
       'KR', 'NGVC', 'QKLS', 'RNDY', 'SFM', 'SFS', 'SWY', 'TFM', 'VLGEA',
       'WFM', 'WMK'], dtype=object)